In [1]:
import os
import os.path as osp
import sys
import logging
import csv
import io
import base64
import ast

from tqdm import tqdm
import json


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas  
from scipy.interpolate import griddata
from scipy.interpolate import RegularGridInterpolator



from flask import Flask, jsonify, request, render_template
from flask_cors import CORS 
from ragutils import ChatGpt, ragChat
from langchain_openai import ChatOpenAI
import importlib
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.documents import Document




In [28]:

modelDict = dict()
with open ('./shapetalk.csv', 'r', encoding='gbk') as f:
    counter = 0
    print('initializing')
    reader = csv.reader(f)
    for row in tqdm(reader):
        if (row[6] not in modelDict):
            modelDict[row[6]] = set()
        for i in range(1, 5):
            if (row[i] != ''):
                modelDict[row[6]].add(row[i])
        counter += 1

modelDict.pop('ShapeNet')
modelDict.pop('source_object_class')
print (modelDict.keys())
embed_model = OpenAIEmbeddings(base_url="https://api3.apifans.com/v1")

vaseDocs = []
vaseId = []
i = 0
# for sentence in list(modelDict['vase']):
#     vaseDocs.append(Document(page_content=sentence))
#     vaseId.append('vase' + str(i))
#     i += 1
for key in modelDict.keys():
    print (key + '-' + str(len(list(modelDict[key]))))

source_knowledge_cap = ''
source_class = 'cap'
for sentence in modelDict['cap']:
    source_knowledge_cap += sentence + '\n'
print(source_knowledge_cap)


initializing


130343it [00:01, 108371.83it/s]


dict_keys(['faucet', 'cap', 'plant', 'cabinet', 'bookshelf', 'knife', 'table', 'skateboard', 'mug', 'vase', 'pistol', 'flowerpot', 'clock', 'chair', 'bathtub', 'bottle', 'display', 'bag', 'trashbin', 'scissors', 'person', 'helmet', 'bowl', 'airplane', 'guitar', 'dresser', 'bed', 'sofa', 'bench', 'lamp'])
faucet-7041
cap-2094
plant-2687
cabinet-2578
bookshelf-8247
knife-4915
table-46480
skateboard-1980
mug-1491
vase-7016
pistol-3660
flowerpot-5625
clock-6116
chair-39479
bathtub-8594
bottle-4149
display-8047
bag-2726
trashbin-4056
scissors-715
person-1768
helmet-1451
bowl-1852
airplane-24645
guitar-6823
dresser-12808
bed-6860
sofa-29684
bench-14045
lamp-36758
It is not a boater hat.
It has a brim and not a visor
It is not a graduation hat (it is a service hat).
target has pyramid crown
The target does not have ear protectors.
It does not have eyelets.
The top is tilted backwards.
does not have a space for a label or badge on it
It does not have a pyramid shaped crown.
target does not hav

In [4]:
# 这里会消耗巨量tokens，慎用
# vectorstore = Chroma.from_documents(documents=vaseDocs, embedding=embed_model, collection_name="model_discription_embed")
# print (vectorstore)

In [14]:
query = "How can I change the vase?"
result = vectorstore.similarity_search(query ,k = 5)
result

NameError: name 'vectorstore' is not defined

In [15]:

ragClient = ragChat()

In [21]:
source_knowledge = 'The target\'s foot is more round and the vase is with builtiful color, the top of it has a longger head\nThe top of the vase is open\nThe vase has flowers in it'
modelClass = 'vase'
prompt_template1 = f"""以下内容为对一个{modelClass}的描述集合，将以下内容中的每一句按照格式以键值字典的方式拆分，键为该物体的部位，如果是对整体的描述则直接用{modelClass}作为键，值为该部位对应的描述。请返回给我一个键值字典列表，列表中每一项为一个键值字典。不要使用代码格式返回:

内容:
{source_knowledge}

"""
ragClient.pushUserMessage(prompt_template1)
answer = ragClient.getGptMsg()
print (answer)


[
  {"foot": "more round"},
  {"vase": "with beautiful color"},
  {"top": "has a longer head"},
  {"top": "is open"},
  {"vase": "has flowers in it"}
]


In [26]:
answer = '''[
  {"foot": "more round"},
  {"vase": "with beautiful color"},
  {"top": "has a longer head"},
  {"top": "is open"},
  {"vase": "has flowers in it"},
  {"top": "has a longer head"}]'''
answer += ''
print (answer)
parsedStr = json.loads(answer)
print (parsedStr)

vasePart2Discription = {}
for dictEle in parsedStr:
    for key in dictEle.keys():
        if key in vasePart2Discription:
            vasePart2Discription[key].add(dictEle[key])
        else:
            vasePart2Discription[key] = set()
            vasePart2Discription[key].add(dictEle[key])
print (vasePart2Discription)

[
  {"foot": "more round"},
  {"vase": "with beautiful color"},
  {"top": "has a longer head"},
  {"top": "is open"},
  {"vase": "has flowers in it"},
  {"top": "has a longer head"}]
[{'foot': 'more round'}, {'vase': 'with beautiful color'}, {'top': 'has a longer head'}, {'top': 'is open'}, {'vase': 'has flowers in it'}, {'top': 'has a longer head'}]
{'foot': {'more round'}, 'vase': {'has flowers in it', 'with beautiful color'}, 'top': {'has a longer head', 'is open'}}


In [24]:
query = '这个花瓶我想让它里面有一束花，我可以如何如何描述这个花瓶？'
prompt_template2 = f"""请基于以下内容回答问题
内容:
{parsedStr}
query:
{query}
"""
ragClient.pushUserMessage(prompt_template2)
answer = ragClient.getGptMsg()
print (answer)

您可以描述这个花瓶为："The vase has flowers in it."
